In [1]:
import os
import re
import json
import tqdm
import torch
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import ray
import openai
import ast
import time
import pickle

os.environ["CUDA_VISIBLE_DEVICES"] = "7"

torch.backends.cuda.matmul.allow_tf32 = True

from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
class RedundantParenthesesRemover(ast.NodeTransformer):
    def visit_Expr(self, node):
        self.generic_visit(node)
        if isinstance(node.value, ast.BinOp):
            return node.value
        return node

def remove_redundant_parentheses(expression):
    # Parse the expression
    parsed_expression = ast.parse(expression)

    # Remove redundant parentheses
    transformer = RedundantParenthesesRemover()
    transformed_expression = transformer.visit(parsed_expression)

    # Convert the transformed expression back to a string
    simplified_expression = ast.unparse(transformed_expression)

    return simplified_expression
    
def divide(a, b):
    return f"({a} / {b})"

def subtract(a, b):
    return f"({a} - {b})"

def multiply(a, b):
    return f"({a} * {b})"

def add(a, b):
    return f"({a} + {b})"

def exp(a, b):
    return f"({a} ** {b})"

def greater(a, b):
    return f"({a} > {b})"

def translate_expr(expr):
    if "table" in expr:
        return expr
    
    # replace const_m1
    expr = re.sub(r'const_m1', r'-1', expr)

    # change % to / 100
    expr = re.sub(r'([0-9]*\.?[0-9]+)%', r'divide(\1 , 100)', expr)
    
    expr = re.sub(r'const_([0-9]*\.?[0-9]+)', r'\1', expr)
    try:
        new_expr = eval(expr)
        new_expr = remove_redundant_parentheses(new_expr)
    except Exception as e:
        print(e, expr)
        new_expr = expr
    
    return new_expr

def convert_to_markdown(data):
    markdown = "|"
    
    # Add table headers
    for header in data[0]:
        markdown += header + "|"
    markdown += "\n|"
    
    # Add table header separators
    for _ in data[0]:
        markdown += "---|"
    markdown += "\n"
    
    # Add table rows
    for row in data[1:]:
        markdown += "|"
        for cell in row:
            markdown += cell + "|"
        markdown += "\n"
        
    return markdown

def extract_answer(response):
    # extract content inside Calculate()
    matches = re.findall(r"Calculate\(([\(\)0-9 ><,\.\/\+\-\*]*)\)", response)
    if len(matches) == 0:
        if "Yes" in response:
            return "Yes"
        elif "No" in response:
            return "No"
        else:
            return ""
    else:
        output = matches[0].replace(",", "")
        return output
    
def if_exec_correct(t_prog, g_prog):
    try:
        t_exec = eval(t_prog)

        if type(t_exec) == bool and g_prog in ["Yes", "No"]:
            t_exec = "Yes" if t_exec else "No"

            if t_exec == g_prog:
                return True
            
        g_exec = eval(g_prog)

        if t_exec == g_exec:
            return True
        elif t_exec * 100 == g_exec:
            return True
        elif t_exec * 100 == -g_exec:
            return True
        elif t_exec == g_exec * 100:
            return True
        elif t_exec == -g_exec * 100:
            return True
        elif t_exec * 1000000 == g_exec:
            return True
        elif t_exec * 1000000 == -g_exec:
            return True
        elif t_exec == g_exec * 1000000:
            return True
        elif t_exec == -g_exec * 1000000:
            return True
        elif t_exec == -g_exec:
            return True
    except:
        return False

    return False

In [3]:
tokenizer = AutoTokenizer.from_pretrained('mosaicml/mpt-7b')

In [4]:
filepath = "../FinQA/dataset/test.json"

with open(filepath) as f:
    data = json.load(f)

programs = []
translated_programs = []
answers = []

for item in tqdm.tqdm(data):
    table_md = convert_to_markdown(item["table_ori"])
    question = item["qa"]["question"]
    
    pre_text = "\n".join(item["pre_text"])
    post_text = "\n".join(item["post_text"])

    programs.append(item["qa"]["program_re"])
    translated_programs.append(translate_expr(item["qa"]["program_re"]))
    answers.append(item["qa"]["answer"])

100%|██████████| 1147/1147 [00:00<00:00, 14480.40it/s]


In [5]:
system_prompt = (
                "You need to answer the user's question in the ### Question ### section.\n" \
                "You need to provide the answer in the format 'Calculate(a + b)', where the expression needs to be python excutable." \
                # "You can calculate the average of a column by using the function 'Average(table_column_name)'.\n" \
                # "Similarly, you can calculate the sum, the maximum, the minimum, the count of a column by using the functions "\
                # "'Sum(table_column_name)', 'Max(table_column_name)', 'Min(table_column_name)', 'Count(table_column_name)' respectively.\n" \
                # "You only use the table's column name inside those operations\n" \
                "For example, if the question is 'What is the sum of 1 + 2?', you need to answer 'Calc(1 + 2)'." \
                "if the question is 'Is 123 greater than 231?', you need to answer 'Calc(123 > 231)'." \
                # "|Age|\n|---|\n|12|\n|15|\n|16|\n\n What is the average age? The answer is 'Calculate(Average(Age))'" \
                "DO NOT give anything else other than'Calculate()'."
                )

In [6]:
import transformers

name = 'mosaicml/mpt-7b'

config = transformers.AutoConfig.from_pretrained(name, trust_remote_code=True)
config.attn_config['attn_impl'] = 'triton'
config.init_device = 'cuda:0' # For fast initialization directly on GPU!

model = transformers.AutoModelForCausalLM.from_pretrained(
  name,
  config=config,
  torch_dtype=torch.bfloat16, # Load model weights in bfloat16
  trust_remote_code=True
)

model = model.eval()

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


You are using config.init_device='cuda:0', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
model = torch.compile(model)

In [8]:
user_example1 = '### Context ###\n\ndebt maturities 2013 the following table presents aggregate debt maturities as of december 31 , 2007 , excluding market value adjustments .\nmillions of dollars .\n\n|2008|$ 689|\n|---|---|\n|2009|542|\n|2010|462|\n|2011|550|\n|2012|720|\n|Thereafter|4,717|\n|Total debt|$ 7,680|\n\n\nat december 31 , 2007 , we reclassified as long-term debt approximately $ 550 million of debt due within one year that we intend to refinance .\nthis reclassification reflected our ability and intent to refinance any short- term borrowings and certain current maturities of long-term debt on a long-term basis .\nat december 31 , 2006 , we did not reclassify any short-term debt as long-term debt as we did not intend to refinance at that mortgaged properties 2013 equipment with a carrying value of approximately $ 2.8 billion at both december 31 , 2007 and 2006 serves as collateral for capital leases and other types of equipment obligations in accordance with the secured financing arrangements utilized to acquire such railroad equipment .\nas a result of the merger of missouri pacific railroad company ( mprr ) with and into uprr on january 1 , 1997 , and pursuant to the underlying indentures for the mprr mortgage bonds , uprr must maintain the same value of assets after the merger in order to comply with the security requirements of the mortgage bonds .\nas of the merger date , the value of the mprr assets that secured the mortgage bonds was approximately $ 6.0 billion .\nin accordance with the terms of the indentures , this collateral value must be maintained during the entire term of the mortgage bonds irrespective of the outstanding balance of such bonds .\ncredit facilities 2013 on december 31 , 2007 , $ 1.9 billion of credit was available under our revolving credit facility ( the facility ) , which we entered into on april 20 , 2007 .\nthe facility is designated for general corporate purposes and supports the issuance of commercial paper .\nwe did not draw on the facility during 2007 .\ncommitment fees and interest rates payable under the facility are similar to fees and rates available to comparably rated , investment-grade borrowers .\nthe facility allows for borrowings at floating rates based on london interbank offered rates , plus a spread , depending upon our senior unsecured debt ratings .\nthe facility requires the maintenance of a debt to net worth coverage ratio .\nat december 31 , 2007 , we were in compliance with this covenant .\nthe facility does not include any other financial restrictions , credit rating triggers ( other than rating-dependent pricing ) , or any other provision that could require us to post collateral .\nthe facility , which expires in april 2012 , replaced two $ 1 billion , 5-year facilities with terms ending in march 2009 and march 2010 .\nthe facility includes terms that are comparable with those of the prior facilities , although the minimum net worth requirement of $ 7.5 billion in prior facilities was removed , and the facility includes a change-of-control provision .\nin addition to our revolving credit facility , a $ 75 million uncommitted line of credit was available .\nthe line of credit expires in april 2008 , and was not used in 2007 .\nwe must have equivalent credit available under our five-year facility to draw on this $ 75 million line .\ndividend restrictions 2013 our revolving credit facility includes a debt-to-net worth covenant that , under certain circumstances , would restrict the payment of cash dividends to our shareholders .\nthe amount of retained earnings available for dividends was $ 11.5 billion and $ 7.8 billion at december 31 , 2007 and december 31 , 2006 , respectively .\nthis facility replaced two credit facilities that had minimum net worth covenants that were more restrictive with respect to the amount of retained earnings available for dividends at december 31 , 2006. .\n\n### Question ###\n\nwhat percentage of total debt is due after 2012?'
assistant_example1 = "### Answer ###\n\nCalculate(4717 / 7680)"

In [9]:
# %%
all_input_ids = []

for item in tqdm.tqdm(data):
    table_md = convert_to_markdown(item["table_ori"])
    question = item["qa"]["question"]
    
    pre_text = "\n".join(item["pre_text"])
    post_text = "\n".join(item["post_text"])
    
    context = f"{pre_text}\n\n{table_md}\n\n{post_text}\n\n"
    user_prompt = f"### Context ###\n\n{context}### Question ###\n\n{question}"
    input_prompt = f"### Instruction ###\n\n{system_prompt}\n\n{user_example1}\n\n{assistant_example1}\n\n{user_prompt}\n\n### Answer ###"

    input_ids = tokenizer.encode(input_prompt, add_special_tokens=False)

    input_ids_length = len(input_ids) 
    max_seq_length = 1984 
    while input_ids_length > max_seq_length:
        # truncate the first input_ids_length - max_seq_length tokens
        context = context.split(" ")[input_ids_length - max_seq_length:]
        context = " ".join(context)
        # recreate the input_text
        user_prompt = f"### Context ###\n\n{context}### Question ###\n\n{question}"
        input_prompt = f"### Instruction ###\n\n{system_prompt}\n\n{user_example1}\n\n{assistant_example1}\n\n{user_prompt}\n\n### Answer ###"

        input_ids = tokenizer.encode(input_prompt, add_special_tokens=False)

        input_ids_length = len(input_ids)

    input_ids = torch.tensor(input_ids)

    all_input_ids.append(input_ids)

100%|██████████| 1147/1147 [00:06<00:00, 171.35it/s]


In [10]:
# %%
tokenizer.pad_token_id = tokenizer.unk_token_id
batch_size = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

responses = []

for i in tqdm.tqdm(range(0, len(all_input_ids), batch_size)):
    batch_input_ids = all_input_ids[i:i+batch_size]
    batch_input_ids = [item.flip(0) for item in batch_input_ids]
    batch_input_ids = pad_sequence(batch_input_ids, 
                                   batch_first=True,
                                   padding_value=tokenizer.pad_token_id).to(device)
    batch_input_ids = batch_input_ids.flip(1)

    attention_mask = (batch_input_ids != tokenizer.pad_token_id).bool().to(device)

    output = model.generate(batch_input_ids,
                            attention_mask=attention_mask,
                            do_sample=False,
                            top_p=0.9, 
                            temperature=0.01, 
                            max_new_tokens=32, 
                            eos_token_id=4118,
                            pad_token_id=tokenizer.pad_token_id, 
                            early_stopping=True)

    for j in range(output.shape[0]):
        input_ids_length = batch_input_ids.shape[1]
        response = tokenizer.decode(output[j][input_ids_length:], skip_special_tokens=True)
        responses.append(response)

  0%|          | 0/287 [00:00<?, ?it/s]/local/data/qywu/PythonLibraries/transformers/src/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
  8%|▊         | 24/287 [05:03<56:32, 12.90s/it]  

In [31]:
result = {
            "responses": responses,
            "model_name": "mpt-7b-zero-shot",
          }

with open("./results/mpt-7b-zero-shot.json", "w") as f:
    json.dump(result, f, indent=4)


In [32]:
gen_programs = []

for response in responses:
    gen_programs.append(extract_answer(response))

In [33]:
# calculate accuracy
prog_correct = 0
exec_correct = 0

wrong_indices = []

for i in range(len(translated_programs)):
    t_prog = translated_programs[i].replace(" ", "")
    g_prog = gen_programs[i].replace(" ", "")

    if t_prog == g_prog:
        prog_correct += 1

    if if_exec_correct(t_prog, g_prog):
        exec_correct += 1
    else:
        wrong_indices.append(i)
        print(i, t_prog, "|", g_prog)

5 (92710000-86842000)/86842000 | (100690000-92710000)/92710000
15 table_average(netchangefortheyear,none) | (4648+3625+1620)/3
18 15.3/(139549/1000) | 15.3/139.5
19 9.7+10.2+(25.0+24.0) | 25.0+9.7
20 (772-843)/843 | (843-772)/772
29 3500/3081 | 3081/(3081+200)
32 table_max(provisionforincometaxes,none) | 
34 61912-367 | 49447-367
47 (559.7-515.2)/559.7 | (765.2-559.7)/559.7
48 10.4/61.4*100 | 10354/61112
59 411636+439000 | (411636+439000+556000)/3
64 194/(2045-1854) | 194/(191*100)
65 28000>19000 | 44000-19000
67 49447/59801 | 49447/60192
70 604.2/8290.6 | 604.2/13440.0
75 11.02/9.52/9.52 | (11.02-9.52)/9.52
78 78227+81808 | 
79 207.62/97.13 | 207.62-100-(97.13-100)
80 207.62-97.13 | 207.62-107.76
83 74360/180993 | 74360/(180993-74360)
84 (1.11+1.08)/1.08 | (1.11-1.08)/1.08
87 2*1000000/2779 | 2779*1000000/411636
88 368-68 | 368/68-1
92 table_average(operatingincome,none) | 
93 426932-341003 | 426932-8.3
95 362400+2000 | 362400+12284
99 7532/180993 | 74360/158578
101 988/3332 | 3332/17

In [34]:
exec_correct / len(translated_programs), prog_correct / len(translated_programs)

(0.6643417611159547, 0.5980819529206626)